In [34]:
#Pkg.add("Combinatorics")
#Pkg.add("Pipe")
#Pkg.add("Lazy")
#Pkg.add("Memoize")

   Updating registry at `C:\Users\Andy\.julia\registries\General`
   Updating registry at `C:\Users\Andy\.julia\registries\JuliaComputingRegistry`
  Resolving package versions...
  Installed Memoize ─ v0.4.4
Updating `C:\Users\Andy\.julia\environments\JuliaPro_v1.5.0-1\Project.toml`
  [c03570c3] + Memoize v0.4.4
Updating `C:\Users\Andy\.julia\environments\JuliaPro_v1.5.0-1\Manifest.toml`
  [c03570c3] + Memoize v0.4.4


In [1]:
using Combinatorics
using Lazy: map, filter, any, all, @as, @>, @>>
using Base.Iterators: flatten, product
using Memoize

In [2]:
mutable struct Node
    coordinates::Tuple{Int,Int}
    cell::Int
    value::Int
    
    possible_values::Vector{Int}
    
    function Node(coordinates::Tuple{Int,Int})
        return new(coordinates,0,0,[])
    end
end

In [3]:
function get_coordinates(node::Node)::Tuple{Int,Int,Int}
    return (node.coordinates..., node.cell)
end

get_coordinates (generic function with 1 method)

In [4]:
function Base.isequal(a::Node,b::Node)
    a,b = get_coordinates.((a,b))
    return all(a.==b)
end

In [9]:
function set_cell!(node::Node, size::Int)::Node
    row, col = node.coordinates .- 1
    node.cell = 1 + floor(((col//size) + row) - (row%size))
    return node
end

set_cell! (generic function with 1 method)

In [10]:
function get_value(node::Node)::Int
    return node.value
end

get_value (generic function with 1 method)

In [11]:
struct Edge
    nodes::Tuple{Node,Node}
    
    function Edge(nodes::Vector{Node})
        return new(tuple(nodes...))
    end
end

In [12]:
function get_nodes(edge::Edge)::Vector{Node}
    return collect(edge.nodes)
end

function get_nodes(edges::Array{Edge})::Vector{Node}
    return map(get_nodes, edges)
end

get_nodes (generic function with 2 methods)

In [13]:
function are_adjacent(edge::Edge)::Bool
    a,b = get_coordinates.(edge.nodes)
    return any(a .== b)
end

are_adjacent (generic function with 1 method)

In [14]:
function Base.in(node::Node, edge::Edge)::Bool
    return any(isequal.(fill(node), edge.nodes))
end

In [15]:
function nodes_to_edges(nodes::Vector{Node})::Vector{Edge}
    return @>> begin
        combinations(nodes,2) 
        map(Edge) 
        filter(are_adjacent)
    end
end

nodes_to_edges (generic function with 1 method)

In [16]:
typeof(Iterators.product(1:3,1:3))

Base.Iterators.ProductIterator{Tuple{UnitRange{Int64},UnitRange{Int64}}}

In [17]:
typeof(1:9)

UnitRange{Int64}

In [18]:
mutable struct SudokuGraph
    nodes::Vector{Node}
    edges::Vector{Edge}
    
    size::Int

    function SudokuGraph(size::Int)        
        #=
        The node coordinates are (row, column, cell),
        generated by creating the cartesian set of (row, col)
        and then assigning the cell based on a formula.
        =#
        
        nodes = @as x begin
            1:size^2
            product(x,x) 
            map(Node, x)
            set_cell!.(x, size) 
            reshape(x, size^4)
        end
        
        edges = nodes_to_edges(nodes)
                
        return new(nodes, edges, size)
    end
end

In [51]:
function get_node(coordinates::Tuple{Int,Int}, graph::SudokuGraph)::Node
    return @>> begin 
        graph.nodes 
        filter(node -> node.coordinates == coordinates) 
        pop!
    end
end

get_node (generic function with 1 method)

In [20]:
function get_neighbors(node::Node, graph::SudokuGraph)::Vector{Node}
    return @>> graph.edges begin
        filter(edge -> node in edge)
        map(get_nodes)
        flatten
        collect
        filter(x -> !isequal(node,x))
    end
end

get_neighbors (generic function with 1 method)

In [21]:
function get_saturated_values(node::Node, graph::SudokuGraph)::Vector{Int}
    return @>> begin
        get_neighbors(node, graph)
        filter(x -> x.value > 0)
        get_value.()
    end
end

get_saturated_values (generic function with 1 method)

In [22]:
function get_saturation(node::Node, graph::SudokuGraph)::Int
    return get_saturated_values(node, graph) |> length
end

get_saturation (generic function with 1 method)

In [23]:
function get_possible_values(node::Node, graph::SudokuGraph)::Vector{Int}
    sv = get_saturated_values(node, graph)
    return collect(filter(x->x ∉ sv, 1:graph.size^2))  
end

get_possible_values (generic function with 1 method)

In [24]:
function set_possible_values!(node::Node, graph::SudokuGraph)::Node
    node.possible_values = get_possible_values(node, graph)
    return node
end

set_possible_values! (generic function with 1 method)

In [25]:
function eliminate_possibility!(node::Node, value::Int)::Node
    if value in node.possible_values
        pop!(node.possible_values, value)
    end
    return node
end

eliminate_possibility! (generic function with 1 method)

In [35]:
function set_value!(node::Node, value::Int)::Node
    node.value = value
    #eliminate_possibility!.(get_neighbors.(node), fill(value))
    return node
end


set_value! (generic function with 1 method)

In [36]:
@timev s = SudokuGraph(3);

  0.001397 seconds (9.97 k allocations: 625.750 KiB)
elapsed time (ns): 1396600
bytes allocated:   640768
pool allocs:       9971
malloc() calls:    2
realloc() calls:   1


In [37]:
length(s.edges)

810

In [38]:
length(s.nodes)

81

In [39]:
a,b = set_cell!.(Node.(((1,1),(1,2))),3)

(Node((1, 1), 1, 0, Int64[]), Node((1, 2), 1, 0, Int64[]))

In [40]:
@timev get_neighbors(a, s);

  0.000039 seconds (839 allocations: 92.391 KiB)
elapsed time (ns): 38700
bytes allocated:   94608
pool allocs:       838
non-pool GC allocs:1


In [41]:
@time get_saturation(a, s)

  0.000073 seconds (841 allocations: 92.703 KiB)


0

In [52]:
set_value!(get_node((5,1), s), 3)

Node((5, 1), 4, 3, Int64[])

In [53]:
@timev get_saturated_values(a, s)

  0.000041 seconds (841 allocations: 92.719 KiB)
elapsed time (ns): 40601
bytes allocated:   94944
pool allocs:       840
non-pool GC allocs:1


1-element Array{Int64,1}:
 3

In [54]:
@show a

a = Node((1, 1), 1, 0, [1, 2, 4, 5, 6, 7, 8, 9])


Node((1, 1), 1, 0, [1, 2, 4, 5, 6, 7, 8, 9])

In [44]:
@timev set_possible_values!(a, s)

  0.000042 seconds (844 allocations: 93.156 KiB)
elapsed time (ns): 42100
bytes allocated:   95392
pool allocs:       843
non-pool GC allocs:1


Node((1, 1), 1, 0, [1, 2, 4, 5, 6, 7, 8, 9])

In [46]:
a

Node((1, 1), 1, 0, [1, 2, 4, 5, 6, 7, 8, 9])